# 📊 **Performance of ReAct Agent Running a Sample Question from TriviaQA Benchmark**


In [4]:
import os
import openai
from agential.cog.agent.react import ReActAgent
from agential.cog.prompts.react import (
    TRIVIAQA_FEWSHOT_EXAMPLES,
    REACT_INSTRUCTION_TRIVIAQA,
)
class GPT3ChatModel:
    def __init__(self, api_key):
        self.api_key = api_key

    def ask(self, prompt, max_tokens=150):
        try:
            response = openai.Completion.create(
                engine="text-davinci-003",  # or choose the specific model version you want
                prompt=prompt,
                max_tokens=max_tokens,
                api_key=self.api_key
            )
            return response.choices[0].text.strip()
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("API key has been activated")


llm = GPT3ChatModel(OPENAI_API_KEY)
print("Initializing the agent")
agent = ReActAgent(llm=llm)

question = "American Callan Pinckney's eponymously named system became a best-selling (1980s-2000s) book/video franchise in what genre?"
outputs = agent.generate(question=question,examples = TRIVIAQA_FEWSHOT_EXAMPLES,
        prompt = REACT_INSTRUCTION_TRIVIAQA)

print(f"Question: {question}") 

if outputs:
    for output in outputs:
        print("Thought:", output.thought)
        print("Action:", output.action)
        print("Observation:", output.observation)
else:
    print("No output generated.")

API key has been activated
Initializing the agent


TypeError: 'GPT3ChatModel' object is not callable